In [1]:
# Install Dependencies
!pip install transformers datasets torch

In [2]:
# Import Required Libraries
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch
from sklearn.model_selection import train_test_split

In [3]:
# Load Pre-trained Model and Tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

In [4]:
# Prepare Dataset
dataset = load_dataset("wikitext", "wikitext-103-raw-v1")
def tokenize_function(examples):
    return tokenizer(examples["text"], return_tensors="pt", truncation=True, padding="max_length", max_length=512)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [5]:
# Split Dataset for Fine-Tuning
train_dataset = tokenized_datasets["train"]
val_dataset = tokenized_datasets["validation"]

In [6]:
# Set Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    evaluation_strategy="steps",
    save_total_limit=2
)

In [7]:
# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

In [8]:
# Fine-Tuning the Model
trainer.train()

In [9]:
# Save the Fine-Tuned Model
trainer.save_model("./fine_tuned_gpt2")
tokenizer.save_pretrained("./fine_tuned_gpt2")

In [10]:
# Advanced Parameter Tuning
training_args = TrainingArguments(
    learning_rate=5e-5,
    warmup_steps=500,
    lr_scheduler_type="linear",
    gradient_accumulation_steps=16,
    fp16=True
)

In [11]:
# Hyperparameter Tuning with Ray Tune (Optional)
from ray import tune
def model_init():
    return GPT2LMHeadModel.from_pretrained("gpt2")
param_space = {
    "learning_rate": tune.loguniform(1e-6, 1e-4),
    "per_device_train_batch_size": tune.choice([4, 8, 16]),
    "num_train_epochs": tune.choice([3, 5, 7]),
}
tune.run(
    lambda config: Trainer(
        model_init=model_init,
        args=TrainingArguments(
            output_dir="./results",
            learning_rate=config["learning_rate"],
            per_device_train_batch_size=config["per_device_train_batch_size"],
            num_train_epochs=config["num_train_epochs"],
        ),
    ).train(),
    config=param_space
)

In [12]:
# Evaluation and Performance Metrics
eval_results = trainer.evaluate()
print(eval_results)